<ul class="breadcrumb">
  <li><a href="2.1.Multivariate_Linear_Mixed_Effects_Model.ipynb">Multivariate Basics</a></li>
  <li><a href="2.2.Multivariate_Linear_Additive_Genetic_Model.ipynb">Multivariate Additive Genetic Model</a></li> 
  <li><a href="2.3.Multivariate_Linear_Mixed_Effects_Model_with_Genomic_Data.ipynb">Multivariate Genomic Data</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multivariate Linear Mixed Effects Model with Genomic Data</font> 
</div>

In [1]:
include("../../src/JWAS.jl")

JWAS

In [2]:
using DataFrames
using JWAS: MT,Datasets,QTL

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [3]:
phenofile = Datasets.dataset("testMT","phenotype.txt")
genofile  = Datasets.dataset("testMT","genotype.txt")
pedfile   = Datasets.dataset("testMT","pedigree.txt");

### phenotypes

In [4]:
;cat $phenofile

Animal,BW,CW,age,sex
S1,100.0,10.0,8,M
D1,50.0,12.9,7,F
O1,150.0,13.0,3,M
O3,40.0,5.0,4,F


### genotypes

In [5]:
;cat $genofile

Animal,x1,x2,x3,x4,x5
S1,1.0,0.0,1.0,1.0,1.0
D1,2.0,0.0,2.0,2.0,1.0
O1,1.0,2.0,0.0,1.0,0.0
O3,0.0,0.0,2.0,1.0,1.0


### pedigree

In [6]:
;cat $pedfile

S1 0 0
D1 0 0
O1 S1 D1
O2 S1 D1
O3 S1 D1


In [7]:
data=readtable(phenofile)

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


<div class="span5 alert alert-info">
 <font size="5" face="Georgia">I. Multiple Traits Analyses with Marker Information</font><br> 
</div>

<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

### Genetic covariance matrix and residual covariance matrix

In [8]:
R      = [10.0 2.0
           2.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

In [9]:
model_equations = "BW = intercept + age + sex;
                   CW = intercept + age + sex";

In [10]:
model = MT.build_model(model_equations,R);

In [11]:
MT.set_covariate(model,"age")

In [12]:
MT.add_markers(model,genofile,G,separator=',',header=true);

The delimiters in file /Users/haocheng/Github/JWAS.jl/src/5.Datasets/src/../data/testMT/genotype.txt is ,  .


<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [13]:
Pi=Dict([1.0; 1.0]=>0.25,[1.0;0.0]=>0.25,[0.0,1.0]=>0.25,[0.0; 0.0]=>0.25)
out = MT.runMCMC(model,data,Pi=Pi,chain_length=500,methods="BayesC",
output_samples_frequency=50);

Priors for marker effects covariance matrix were calculated from genetic covaraince matrix and π.
Marker effects covariance matrix is 
[17.53 1.75
 1.75 1.75].


running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:01


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [14]:
keys(out)

Base.KeyIterator for a Dict{Any,Any} with 5 entries. Keys:
  "Posterior mean of residual covariance matrix"
  "Posterior mean of marker effects"
  "Posterior mean of marker effects covariance matrix"
  "MCMC samples for residual covariance matrix"
  "Posterior mean of location parameters"

In [15]:
file1="marker_effects_BW_2016-07-29T12:51:23.txt"
file2="marker_effects_CW_2016-07-29T12:51:23.txt";

In [16]:
samples4G,dict4G=get_additive_genetic_variances(model,file1,file2);

In [17]:
samples4R=out["MCMC samples for residual covariance matrix"];

In [18]:
samples4h2,dict4h2=get_heritability(samples4G,samples4R);

In [28]:
keys(dict4h2)

Base.KeyIterator for a Dict{ASCIIString,Array{Float64,2}} with 2 entries. Keys:
  "mean_of_heritability"
  "variance_of_heritability"

In [29]:
dict4h2["mean_of_heritability"]

2x2 Array{Float64,2}:
 0.799966  0.59977
 0.59977   0.81131

<div class="span5 alert alert-info">
 <font size="5" face="Georgia">II. Multiple Traits Analyses with Marker Effects and Polygenic Effects</font><br> 
</div>

<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

#### set up model equations

In [19]:
model_equations = "BW = intercept + age + sex + Animal;
                   CW = intercept + age + sex + Animal";

In [20]:
model4    = MT.build_model(model_equations,R);

In [21]:
MT.set_covariate(model4,"age")

In [22]:
GA = G*0.1
MT.set_random(model4,"Animal",ped,GA)

LoadError: LoadError: UndefVarError: ped not defined
while loading In[22], in expression starting on line 2

In [23]:
GM = G*0.9
MT.add_markers(model4,genofile,GM,separator=',',header=true);

The delimiters in file /Users/haocheng/Github/JWAS.jl/src/5.Datasets/src/../data/testMT/genotype.txt is ,  .


<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [24]:
Pi=Dict([1.0; 1.0]=>0.25,[1.0;0.0]=>0.25,[0.0,1.0]=>0.25,[0.0; 0.0]=>0.25)
out4=MT.runMCMC(model4,data,Pi=Pi,chain_length=500,printout_frequency=300,methods="BayesC");

Priors for marker effects covariance matrix were calculated from genetic covaraince matrix and π.
Marker effects covariance matrix is 
[15.78 1.58
 1.58 1.58].



Posterior means at iteration: 300
Residual covariance matrix: 
[30.67 5.289
 5.289 1.527]
Marker effects covariance matrix: 
[11.343 1.0
 1.0 1.301]

running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:00


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [25]:
keys(out4)

Base.KeyIterator for a Dict{Any,Any} with 4 entries. Keys:
  "Posterior mean of residual covariance matrix"
  "Posterior mean of marker effects"
  "Posterior mean of marker effects covariance matrix"
  "Posterior mean of location parameters"

In [26]:
#out4["Posterior mean of polygenic effects covariance matrix"]
#out4["Posterior mean of location parameters"]
#out4["Posterior mean of marker effects"][1]
out4["Posterior mean of marker effects"][2]

5x2 Array{Any,2}:
 "x1"  -0.0874222
 "x2"  -0.0180527
 "x3"  -0.0299907
 "x4"   0.101756 
 "x5"   0.0257107